<a href="https://colab.research.google.com/github/AnshUpadhyay639/LLM-Decoder-Prototype/blob/main/LLM_Decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a GPT

In [ ]:
# Download the dataset:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-02-03 18:18:23--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.08s   

2025-02-03 18:18:23 (12.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read() # Stores all the contents of the file as a List of Strings.

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [ ]:
# first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
# Create a Vocab Size:
chars = sorted(list(set(text)))
vocab_size = len(chars) # No. of Unique Characters.
print(' '.join(chars))
print(vocab_size)


   ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z
65


In [ ]:
# Create a mapping from characters to integers:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string = s as input, output a list of integers.
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers = l as input, output a string.

print(encode("Hello World"))
print(decode(encode("Hello World")))

[20, 43, 50, 50, 53, 1, 35, 53, 56, 50, 42]
Hello World


In [ ]:
# Convert the entire text dataset and store it into a torch.Tensor:
import torch
data = torch.tensor(encode(text), dtype=torch.long) # We convert the encoded text ofc.
print(data.shape, data.dtype) # No shape till now whatsoever [Raw Data Corpus].
print(data[:1000]) # The 1000 characters we looked at earier will to the GPT look like this.

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [ ]:
# Split up the data into train and validation sets:
n = int(0.9*len(data)) # first 90% will be train, rest val.
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8 # The No. of characters used at once to train the Model [No. of Features of a single input data].
train_data[:block_size+1] # 9th character is the answer to the first 8 chars.[We be used when splitting x and y].

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x = train_data[:block_size] # Contains 8 inputs and for each single input char, its o/p lies in the idx next to it.
y = train_data[1:block_size+1] # Offset of index 1 for all inputs.
for t in range(block_size):
    context = x[:t+1] # Random Numbers selected.
    target = y[t] # The answer to that context.
    print(f"when input is {context} the target: {target}") # Look at the last line ; Hence we needed 9 elements to train single data row of (8 ele).

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel? [No. of rows]
block_size = 8 # what is the maximum context length for predictions? [No. of features for a single row][No. of Cols].

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) # Create 4 random indexes in range(len-8, 8).[len-8 == No ArrayIndexOutOfBounds].
    x = torch.stack([data[i:i+block_size] for i in ix]) # Create 4 random data rows and stack them on top of each other.
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # Vice-versa.
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    print(f"For Row {b}:")
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
For Row 0:
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
For Row 1:
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46

In [ ]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module): # Implements a simple Bigram Model.

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table:
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # Create a lookup table.

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)[Batch, Time stamp(for serial data)= num_features, Channels = Vocab Size][Stores the scores of next character in the sequence].

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # [4, 8, 65].
            logits = logits.view(B*T, C) # For inputs to the loss function.
            targets = targets.view(B*T) # Do the same for targets.
            loss = F.cross_entropy(logits, targets) # Negative Log-likelihood loss.[Accpets Channels to be 2nd dim and rest first].

        return logits, loss

    def generate(self, idx, max_new_tokens): # max_new_tokens = Max no. of new tokens it can generate with input as idx[4,8].
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx) # self.forward()
            # focus only on the last time step.
            logits = logits[:, -1, :] # Excludes the last prediction value (-1) from the Time dimension. Now becomes Shape: [B, C].
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # Shape: (B, 1)[For each Batch dim "B" we will have a single prediction].
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # Shape: (B, T+1) # Concatenates the Predicted idx after the OG idx and returns the new idx.
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100).squeeze().tolist())) # idx[B=1,T=1] = 0.


torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [ ]:
# Create an Optimizer:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(1000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


3.7218432426452637


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


olylvLLko'TMyatyIoconxad.?-tNSqYPsx&bF.oiR;BD$dZBMZv'K f bRSmIKptRPly:AUC&$zLK,qUEy&Ay;ZxjKVhmrdagC-bTop-QJe.H?x
JGF&pwst-P sti.hlEsu;w:w a BG:tLhMk,epdhlay'sVzLq--ERwXUzDnq-bn czXxxI&V&Pynnl,s,Ioto!uvixwC-IJXElrgm C-.bcoCPJ
IMphsevhO AL!-K:AIkpre,
rPHEJUzV;P?uN3b?ohoRiBUENoV3B&jumNL;Aik,
xf -IEKROn JSyYWW?n 'ay;:weO'AqVzPyoiBL? seAX3Dot,iy.xyIcf r!!ul-Koi:x pZrAQly'v'a;vEzN
BwowKo'MBqF$PPFb
CjYX3beT,lZ qdda!wfgmJP
DUfNXmnQU mvcv?nlnQF$JUAAywNocd  bGSPyAlprNeQnq-GRSVUP.Ja!IBoDqfI&xJM AXEHV&DKvRS


## The mathematical trick in self-attention

In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation".
# Fast Method for avergaing||mean to built relation b/w curr T and all the prev T to predict the future T.
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3)) # Only Left Lower Triangle all 1s ; Now if c = a @ b then c will just be the dynamically accumulated rows of matrix "b".
a = a / torch.sum(a, 1, keepdim=True) # If we mean() all the a.values along the columns then c will also contain the dynamically accumulated mean of matrix "b".
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c) # Conatins the Column wise mean of matrix b.

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# consider the following toy example:
torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels [each token = time dim].
x = torch.randn(B,T,C)
x.shape # Now we want to make a relationship b/w token 5(say) with all the previous tokens to predict token 6.

torch.Size([4, 8, 2])

In [ ]:
# Slow Method: We want x[b,t] = mean_{i<=t} x[b,i] to make a relation b/w prev tokens and the curr token by averaging.
xbow = torch.zeros((B,T,C)) # Bag of Words.
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # Shape: (t,C) contains all the previous T values upto this point.
        xbow[b,t] = torch.mean(xprev, dim=0) # Mean with the rows and replace with the original row with row mean.
x[0], xbow[0]

(tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [ ]:
# version 2: using matrix multiply for a weighted aggregation [Implementing the Faster Method to Calculate the Mean].
wei = torch.tril(torch.ones(T, T)) # This results in a [T, T] attention matrix, meaning every token attends to all other tokens at the same time.
wei = wei / wei.sum(1, keepdim=True) # Calculating the Mean of each row by div. it by the sum of that particular row.
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C) [Boradcasting happens for "wei"].
torch.equal(xbow, xbow2)
x[0], xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [ ]:
# version 3: use Softmax [Even Better version]
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # Where there is a 0 change it to -inf in wei so that e^x of that in softmax = 0.[Doing this tells the model to not mul the future matrix with the curr one; Only mul with past values].
wei = F.softmax(wei, dim=-1) # Calculate mean() of exps.
xbow3 = wei @ x # Same results.
torch.allclose(xbow2, xbow3)

True

In [ ]:
# version 4: self-attention! [Input Seq or Token --> 3 Vectors[Query, Key, Value]].[Query = What this token wants][Key = What the token contains].
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels.
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16 # Hyperparameter.[In MHA: We split the input embeddings into "n" different heads with each head having a size].
key = nn.Linear(C, head_size, bias=False) # (4, 8, 32)--->(4, 8, 16).[nn.Linear(): Treats the Last dim as num_features and the rest it treats & ignores as batch dimensions].
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # [-1 = Last dim && -2 = 2ndLast dim] (B, T, 16) @ (B, 16, T) ---> (B, T, T) [Wei now computes attention scores, which determine how much focus one token should have on others in a sequence].
#If the query and key are similar (i.e., they align in the vector space), their dot product will be higher, meaning the query "pays more attention" to that key.
tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # Avoid Interaction with forward values by making the upper matrix all zeros.
wei = F.softmax(wei, dim=-1) # Perform Softmax over Last Dimension i.e(Over all the T values of wei[B,T,T]).

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space (like in CNN). Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:
k = torch.randn(B,T,head_size) # dim of keys = head_size rn
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5 # Scaled Dot Product Attention.[Otherwise Softmax graph will be peaky i.e.(Aggregates all info in a single node)].

In [ ]:
k.var() # Gaussian Variance (i.e var == 1).

tensor(1.0449)

In [ ]:
q.var() # Gaussian Variance

tensor(1.0700)

In [ ]:
wei.var() # Becomes Gauss Var() after scaling through div. by (dim 0f keys**0.5).

tensor(1.0918)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0 # P(a) for a neuron to be set to zero.
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """Implements one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size))) # Assigns the Lower Triangular Matrix to a Buffer named 'tril'.

        self.dropout = nn.Dropout(dropout) # Randomly drop neurons to zero to prevent Overfitting.[This Layer doesnt work during Training with Inference Mode].

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # Compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T) [DECODER BLOCK].
        wei = F.softmax(wei, dim=-1) # (B, T, T) [Apply Softmax at the last dim = -1].
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size): # How many heads u want ; Size of each Head.
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)]) # Create a List of "n" Heads with head_size [nn.ModuleList() is used to store a List of any Modules(nn.Layers, self made classes, etc)].
        self.proj = nn.Linear(n_embd, n_embd) # Linear Transforms the Final Output.
        self.dropout = nn.Dropout(dropout)

    def forward(self, x): # Each head looks over the entire input sequence and has its unique outputs [as nn.Linear() has Random Wt. Matrix.]
        out = torch.cat([h(x) for h in self.heads], dim=-1) # Concatenates the O/ps of all the heads
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module): # Processes a Single Token instead of looking at all the tokens in a sentence at once (like MHA).[Introduces Non-Linearity and gives our model more time to think for a single token].
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd): # A single Token has Shape: (n_embd) after Normalization.
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd), # [Hidden Units = Number of Output Features = 4 * n_embd].
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module): # Just as CNN has Conv Blocks, Transformer also has Blocks where each Block performs (MHA-->Norm-->FFN).
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like.
        super().__init__()
        head_size = n_embd // n_head # In General.
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # Residual Connections [x = x(Direct Path) + x'(Layered Path)].
        x = x + self.ffwd(self.ln2(x)) # Notice that LayerNorm is Applied before any Computations for Optimization.
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd) # row = each character && col = No. of feature of each char.
        self.position_embedding_table = nn.Embedding(block_size, n_embd) # new row = block size.[Which stores the indexes of the incomming chars and represents each index in 4 dims so that they can be added directly with the Token Embeddings].
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # Layer norm (Normalize along the colums (dim=1) i.e Looks at all the features simultaneously)[Opp. of BatchNorm(dim=0)].
        self.lm_head = nn.Linear(n_embd, vocab_size) # Last Layer Ouput = Num_Classes = Vocab_size.

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C) Token embeddings represent the semantic meaning of words.
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C) Positional embeddings encode the position of each token in a sentence, helping the model understand word order.
        x = tok_emb + pos_emb # (B,T,C) Addition allows the model to learn a smooth representation rather than treating position as a separate feature.
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:] # So that Postional embedding matrix doesnt get ArrayIndexOutOfBounds.
            # get the predictions
            logits, loss = self(idx_cond) # Calls def forward().
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 4: train loss 3.7898, val loss 3.8054

hRAkuX vM S &rrl'xf;f'pXMHuVwKQb XfXaei Ktztwrh'Ve ioHfi jK txpabtiK! ; eH;z$e!ALYa?Kf e ;StchiBqfENn fyRGmcueptN s&tw
$Kpm
curMYVOnB:mi,reqfBr lL B f sct
teN.3tQle cn! boXf;$hk'tr,uQbfK3opKlFr joU?iccwioiB,;ee$ raGrlentlrei-cQit DecfV avaae$Yr'fUjXh,?heiKPBMv?' 
yiOIUyc'DM t
BsciH kyZrbGr3Qp,YsQi,pKA:Q,r$gsrmcwLvYT'iyrnrC
DSicb'cy . VyKhfVTemieMfcTbH.r  Rks eYF okR,.a3i pHEir DhCiitirhURi'Hbbve ciJdqrfwcttpRdr'en?z-gfFgM'rrRn
 fRdfm UeiH lf'thded lAOd$y'FQ'AG 
XaW'oLw -rAgrsTl?
rhPeVIg?BV ;zcumw. WihmtUlrs!?3o$fYkDyiUeEa'Fl!cr fzM.E!JQe qir'.teCiIY rDemReeLwvU-aUrC !ffFbfcfedKOR eXz$
 JPh:ean? St.  B:f VUr' VTVcykeDt'
i,pc;UqU$rHUo''
 lzWat? 
 Meui' pawSzfH?K KReH iKe-ewiti kh ! ;zieKXy l &o'ff rRKch3kShh.ws:WD l.n ed FrfUd?$c3 UonrPfIuHnkl oRzewuf.IReMzVw qpeLG;hoea&'c'&XKC HEfEHHA;BUpYe'nVoCRJql zXb:!?r
r-QY caMgorK seziH U&nM U & bf ptL:ry m$,;eDz,'rmaXTerht'cPlQk?EyT$Qefo$K